<a href="https://colab.research.google.com/github/achmadgani/devops_training/blob/main/colletothricum_growth_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INTRODUCTION TO PYTHON

# Introduction

### Program Explanation

# Colletothricum Clades Implementation

Code for Colletothricum Clades implementation python Program

In [9]:
import pandas as pd
import numpy as np

class ColletotrichumGrowthCalculator:

    def __init__(self, csv_path,HS=1, r=0.05, p=6, i=28, RcOPT=0.1):
        self.csv_path = csv_path
        self.HS = HS
        self.r = r
        self.p = p
        self.i = i
        self.RcOPT = RcOPT
        self.df = pd.read_csv(csv_path)
        self.rename_columns()

    def clamp(self,value):
        return max(0, min(value, 1))

    def rename_columns(self):
        # Rename columns by removing any parentheses and the content within them
        self.df.columns = [col.split('(')[0].strip() for col in self.df.columns]

    def primary_inoculum(self):
        # Assuming the DataFrame is already loaded and columns are renamed

        # f(WD)
        self.df['f_WD'] = np.where(self.df['wetness_duration'] >= 3,
                                    np.exp(-5.947 * np.exp(-0.067 * self.df['wetness_duration'])), 0)

        # Teq
        self.df['Teq'] = np.where((self.df['temperature'] > 40) | (self.df['temperature'] < 5),
                                   0, (self.df['temperature'] - 5) / (40 - 5))

        # f(T)
        self.df['f_T'] = (4.975 * self.df['Teq']**1.354 * (1 - self.df['Teq']))**9.409

        # f1(WD)
        self.df['f1_WD'] = np.where(self.df['wetness_duration'] >= 3,
                                     0.398449 * np.exp(-0.067 * self.df['wetness_duration']) -
                                     5.947 * np.exp(-0.067 * self.df['wetness_duration']), 0)

        # SPOR'
        self.df['SPOR_prime'] = 0.00004 * self.df['f_T'] * self.df['f1_WD']

        # Initialize CON column with zeros
        self.df['CON'] = 0

        # Loop through the DataFrame rows to calculate CON values
        for i in range(1, len(self.df)):
            self.df.loc[i, 'CON'] = self.df.loc[i - 1, 'CON'] + self.df.loc[i, 'SPOR_prime']

    def conidia_dispersal(self):
        # DISR_f1(P)
        self.df['DISR_f1_P'] = np.where(self.df['precipitation'] >= 0.3,
                                         np.exp(-2.5 * np.exp(-0.15 * self.df['precipitation'])), 0)

        # DISR_f2(P)
        self.df['DISR_f2_P'] = np.where(self.df['precipitation'] > 15,
                                         1 / (1 + 966.9 * np.exp(-0.133 * self.df['precipitation'])), 0)

        # DISR
        self.df['DISR'] = np.where(self.df['precipitation'] >= 0.3,
                                    self.df['DISR_f1_P'] * (1 - self.df['DISR_f2_P']), 0)


    def conidia_infection(self):
        # Calculate RcWD
        self.df['RcWD'] = 1 - np.exp(-0.1 * self.df['wetness_duration'])

        # Calculate RcWD'
        self.df['RcWD_prime'] = 0.1 * np.exp(-0.1 * self.df['wetness_duration'])

        # Calculate Teq(Rc)
        self.df['Teq_Rc'] = np.where(
            (self.df['temperature'] > 33) | (self.df['temperature'] < 5),
            0,
            (self.df['temperature'] - 5) / (33 - 5)
        )

        # Calculate RcT
        self.df['RcT'] = (5.908 * self.df['Teq_Rc'] ** 1.689 * (1 - self.df['Teq_Rc'])) ** 5.523

        # Calculate Rc using the constant RcOPT
        self.df['Rc'] = self.RcOPT * self.df['RcT'] * self.df['RcWD_prime']


    def incubation_latency_period(self):
        # Constants for the formulas
        Tmin_Inc = 5
        Topt_Inc = 24.2
        Tmax_Inc = 30.5
        LPmin = 140.4  # Added LPmin constant


        # Calculations for incubation (I)
        self.df['Ta'] = self.df['temperature'] - Tmin_Inc
        self.df['Tb'] = Topt_Inc - Tmin_Inc
        self.df['Tc'] = Tmax_Inc - self.df['temperature']
        self.df['Td'] = Tmax_Inc - Topt_Inc
        self.df['Tc_Td'] = self.df['Tc'] / self.df['Td']
        self.df['Te'] = (Tmax_Inc - Topt_Inc) / (Topt_Inc - Tmin_Inc)
        self.df['Ta_Tb'] = self.df['Ta'] / self.df['Tb']
        self.df['Tc_Td_Te'] = self.df['Tc_Td'] ** self.df['Te']
        self.df['f_T_I'] = self.df['Ta_Tb'] * self.df['Tc_Td_Te']

        # Constants for the latency period (L)
        Tmin_Lat = 2  # Assuming these are constants provided for latency period
        Topt_Lat = 28.8
        Tmax_Lat = 40
        IPmin = 118.3  # Incubation period minimum constant

        # Calculations for latency period (L)
        self.df['Taa'] = self.df['temperature'] - Tmin_Lat
        self.df['Tbb'] = Topt_Lat - Tmin_Lat
        self.df['Tcc'] = Tmax_Lat - self.df['temperature']
        self.df['Tdd'] = Tmax_Lat - Topt_Lat
        self.df['Tcc_Tdd'] = self.df['Tcc'] / self.df['Tdd']
        self.df['Tee'] = (Tmax_Lat - Topt_Lat) / (Topt_Lat - Tmin_Lat)
        self.df['Taa_Tbb'] = self.df['Taa'] / self.df['Tbb']
        self.df['Tcc_Tdd_Tee'] = self.df['Tcc_Tdd'] ** self.df['Tee']
        self.df['f_T_L'] = self.df['Taa_Tbb'] * self.df['Tcc_Tdd_Tee']

        # pi and pl calculation
        self.df['pi'] = self.df['f_T_I'] / IPmin
        self.df['pl'] = self.df['f_T_L'] / LPmin

    def rate_variables(self):
      # This function calculates the various rates based on the current state of the system

      # Constants
      AA = 1  # As per the given formula, might represent a scaling factor or similar

      # Rate Variables
      self.df['INFR'] = AA * self.df['CON'] * self.df['DISR'] * self.df['Rc'] * 36000
      self.df['INCR'] = AA * self.df['f_T_L'] * self.df['temperature'] * self.df['CON']
      self.df['LATR'] = AA * self.df['Tdd'] * self.df['f1_WD'] * 0.0001
      self.df['REMR'] = 0

    def state_variables(self):
      # Initialize state variables if they don't exist
      if 'H' not in self.df.columns:
          self.df['H'] = 0  #
          self.df['L'] = 0  #
          self.df['V'] = 0  #
          self.df['I'] = 0

      # Update state variables
      for t in range(1, len(self.df)):
          # H at time t = H at time t-1 minus INFR at time t multiplied by a constant (assumed 1 here)
          self.df.loc[t, 'H'] = self.df.loc[t-1, 'H'] - self.df.loc[t, 'INFR'] * 1
          new_H = self.df.loc[t-1, 'H'] - self.df.loc[t, 'INFR'] * 1
          new_L = self.df.loc[t-1, 'L'] + self.df.loc[t, 'INFR'] * 1 - self.df.loc[t, 'INCR'] * 1
          new_V = self.df.loc[t-1, 'V'] + self.df.loc[t, 'INCR'] * 1 - self.df.loc[t, 'LATR'] * 1
          new_I = self.df.loc[t-1, 'I'] + self.df.loc[t, 'LATR'] * 1
          new_R = 0
          new_D = new_V + new_I + new_R

          # Clamp values to the range [0, 1]
          self.df.loc[t, 'H'] = self.clamp(new_H)
          self.df.loc[t, 'L'] = self.clamp(new_L)
          self.df.loc[t, 'V'] = self.clamp(new_V)
          self.df.loc[t, 'I'] = self.clamp(new_I)
          self.df.loc[t, 'R'] = self.clamp(new_R)
          self.df.loc[t, 'D'] = self.clamp(new_D)

    # def combine_results(self):
    #     # This combines all the results into a single DataFrame
    #     # You could select only the columns with the results
    #     relevant_columns = ['conidia_dispersal_result', 'conidia_infection_result', 'incubation_latency_period_result']
    #     # Add more columns from self.df as needed
    #     self.mathematical_result = self.df[relevant_columns].copy()

    # def calculate_growth(self):
    #     # Add the calculation logic here
    #     self.df['growth'] = self.df.apply(lambda row: self.formula(row), axis=1)

    # def formula(self, row):

    def save_results(self, output_path):
        self.df.to_csv(output_path, index=False)

    def display_dataframe(self):
    # Use the built-in display function to nicely format the output in Jupyter/Colab
        display(self.df)

# Usage:
# Initialize the calculator with the path to your CSV file
calculator = ColletotrichumGrowthCalculator('/content/mulberry - tsukuba data.csv')
calculator.primary_inoculum()
calculator.conidia_dispersal()
calculator.conidia_infection()
calculator.incubation_latency_period()
calculator.rate_variables()
calculator.state_variables()

# Combine all the results into the mathematical_result DataFrame
# calculator.combine_results()

# Now you can access the combined results with:
# mathematical_result = calculator.mathematical_result

# Assuming 'calculator.df' is your DataFrame
# First, get a list of all the current column names
columns = list(calculator.df.columns)

# Define the columns you want to move
columns_to_move = ['H', 'L', 'V', 'I', 'R']

# Remove these columns from the original list
for col in columns_to_move:
    columns.remove(col)

insert_position = 10

# Reinsert the columns at the desired position
new_columns = columns[:insert_position] + columns_to_move + columns[insert_position:]

# Reorder the DataFrame columns
calculator.df = calculator.df[new_columns]

calculator.display_dataframe()
# calculator.save_results('mathematical_result.csv')


,datetime,temperature,precipitation,humidity,wetness_duration,D,f_WD,Teq,f_T,f1_WD,...,Tee,Taa_Tbb,Tcc_Tdd_Tee,f_T_L,pi,pl,INFR,INCR,LATR,REMR
0,1993/7/1 1:00,16.5,0.0,81,0,0.0,0.000000,0.328571,0.058930,0.000000,...,0.41791,0.541045,1.363027,0.737459,0.006580,0.005253,0.0,0.000000,0.000000,0
1,1993/7/1 2:00,16.9,0.0,79,0,0.0,0.000000,0.340000,0.077510,0.000000,...,0.41791,0.555970,1.353283,0.752385,0.006744,0.005359,0.0,0.000000,0.000000,0
2,1993/7/1 3:00,16.6,0.0,80,0,0.0,0.000000,0.331429,0.063214,0.000000,...,0.41791,0.544776,1.360600,0.741222,0.006621,0.005279,0.0,0.000000,0.000000,0
3,1993/7/1 4:00,15.1,0.0,87,0,0.0,0.000000,0.288571,0.019434,0.000000,...,0.41791,0.488806,1.396391,0.682564,0.005962,0.004862,0.0,0.000000,0.000000,0
4,1993/7/1 5:00,15.8,0.0,85,0,0.0,0.000000,0.308571,0.034899,0.000000,...,0.41791,0.514925,1.379850,0.710520,0.006279,0.005061,0.0,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3643,1993/11/29 20:00,20.2,0.0,92,8,0.0,0.030823,0.434286,0.410819,-3.246368,...,0.41791,0.679104,1.268851,0.861683,0.007863,0.006137,-0.0,-0.815185,-0.003636,0
3644,1993/11/29 21:00,20.4,0.0,91,8,0.0,0.030823,0.440000,0.441053,-3.246368,...,0.41791,0.686567,1.263479,0.867463,0.007916,0.006179,-0.0,-0.829792,-0.003636,0
3645,1993/11/29 22:00,20.7,0.0,92,8,0.0,0.030823,0.448571,0.487762,-3.246368,...,0.41791,0.697761,1.255361,0.875942,0.007991,0.006239,-0.0,-0.851373,-0.003636,0
3646,1993/11/29 23:00,20.7,0.0,92,8,0.0,0.030823,0.448571,0.487762,-3.246368,...,0.41791,0.697761,1.255361,0.875942,0.007991,0.006239,-0.0,-0.852522,-0.003636,0


In [ ]:
self.df['CON']

NameError: name 'self' is not defined